In [7]:
import string
from pathlib import Path
from collections import defaultdict
import json
import numpy as np
import math
from tqdm import tqdm

file = open('./input.txt', 'r')

blueprints = {}

for line in file.readlines():
    line = line.replace('\n','')

    blueprint_id, costs = line.split(':')
    blueprint_id = int(blueprint_id.replace('Blueprint ',''))

    costs = costs.split('.')
    costs = [cost.split('costs')[1] for cost in costs if len(cost)>0]

    ore_cost = int(costs[0].replace('ore','').strip())
    clay_cost = int(costs[1].replace('ore','').strip())
    obs_cost = costs[2].split(' and ')
    obs_cost = (int(obs_cost[0].replace('ore','').strip()), int(obs_cost[1].replace('clay','').strip()))
    geode_cost = costs[3].split(' and ')
    geode_cost = (int(geode_cost[0].replace('ore','').strip()), int(geode_cost[1].replace('obsidian','').strip()))

    blueprints[blueprint_id] = (ore_cost, clay_cost, obs_cost[0], obs_cost[1], geode_cost[0], geode_cost[1])

blueprints
# print(json.dumps(blueprints, indent=4))

{1: (4, 4, 4, 5, 3, 7),
 2: (4, 4, 4, 15, 4, 17),
 3: (4, 4, 2, 9, 3, 15),
 4: (4, 4, 2, 8, 3, 9),
 5: (3, 3, 2, 19, 2, 12),
 6: (2, 3, 2, 14, 3, 8),
 7: (4, 4, 4, 12, 4, 19),
 8: (4, 3, 3, 10, 3, 10),
 9: (2, 3, 3, 9, 3, 9),
 10: (4, 4, 2, 11, 4, 8),
 11: (4, 4, 2, 18, 4, 20),
 12: (4, 3, 4, 20, 4, 8),
 13: (2, 4, 3, 19, 4, 13),
 14: (4, 4, 3, 19, 4, 15),
 15: (4, 4, 4, 7, 2, 16),
 16: (3, 4, 2, 20, 4, 7),
 17: (4, 3, 4, 15, 4, 9),
 18: (3, 4, 3, 6, 4, 11),
 19: (4, 4, 4, 17, 4, 16),
 20: (2, 2, 2, 8, 2, 14),
 21: (2, 3, 3, 13, 3, 15),
 22: (3, 3, 3, 9, 2, 10),
 23: (3, 4, 4, 18, 2, 11),
 24: (2, 3, 2, 16, 2, 9),
 25: (4, 4, 3, 6, 2, 14),
 26: (4, 4, 4, 8, 4, 14),
 27: (3, 3, 3, 9, 3, 7),
 28: (4, 3, 3, 11, 4, 7),
 29: (2, 4, 2, 16, 4, 12),
 30: (4, 3, 2, 20, 3, 9)}

In [8]:
from collections import deque
def solve(costs, initial_state=(1,0,0,0,0,0,0,0,24)):
    reached_states = deque([initial_state])
    seen = set()
    best = (0,24)
    best_state = initial_state
    coo, cco, cOo, cOc, cgo, cgO = costs
    max_co = max([coo,cco,cOo,cgo])
    max_cc = cOc
    max_cO = cgO

    while reached_states:
        # print(reached_states)
        state = reached_states.popleft()
        (r1,r2,r3,r4,res1,res2,res3,res4,t) = state
        if res4 >= best[0]:
            best = (res4,t)
            best_state = (r1,r2,r3,r4,res1,res2,res3,res4,t)
        if t==0:
            continue

        r1 = min(r1, max_co)
        r2 = min(r2, max_cc)
        r3 = min(r3, max_cO)
        res1 = min(res1, (t)*max_co)
        res2 = min(res2, (t)*max_cc)
        res3 = min(res3, (t)*max_cO)

        res_state = (r1,r2,r3,r4,res1,res2,res3,res4)
        if res_state in seen:
            continue
        seen.add(res_state)
        

        collecting_state = (r1,r2,r3,r4,res1+r1,res2+r2,res3+r3,res4+r4,t-1)
        reached_states.append(collecting_state)

        if res1>=coo:
            built_state = (r1+1,r2,r3,r4,res1+r1-coo,res2+r2,res3+r3,res4+r4,t-1)
            reached_states.append(built_state)
        if res1>=cco:
            built_state = (r1,r2+1,r3,r4,res1+r1-cco,res2+r2,res3+r3,res4+r4,t-1)
            reached_states.append(built_state)
        if res1>=cOo and res2>=cOc:
            built_state = (r1,r2,r3+1,r4,res1+r1-cOo,res2+r2-cOc,res3+r3,res4+r4,t-1)
            reached_states.append(built_state)
        if res1>=cgo and res3>=cgO:
            built_state = (r1,r2,r3,r4+1,res1+r1-cgo,res2+r2,res3+r3-cgO,res4+r4,t-1)
            reached_states.append(built_state)

            
    return best, best_state


In [9]:
max_minutes = 24
initial_state = (1,0,0,0,0,0,0,0, max_minutes)
reached_states = []
quality_level = 0
blueprints_bests = {}
for blueprint_id, costs in tqdm(blueprints.items()):

    best, best_state = solve(costs, initial_state=initial_state)

    blueprints_bests[blueprint_id] = (best, best_state)
    quality_level += blueprint_id*best[0]
        
print(f'QUALITY LEVEL : {quality_level}')
for blueprint_id, (best,best_state) in tqdm(blueprints_bests.items()):
    print(f'for blueprint {blueprint_id} best is to collect {best[0]} geodes with {best[1]} minutes left')
    print(f'quality = {blueprint_id*best[0]}')
    print(f'final state is {best_state}')

100%|██████████| 30/30 [00:13<00:00,  2.17it/s]


QUALITY LEVEL : 1981


100%|██████████| 30/30 [00:00<00:00, 57693.31it/s]

for blueprint 1 best is to collect 9 geodes with 0 minutes left
quality = 9
final state is (3, 3, 4, 3, 6, 8, 9, 9, 0)
for blueprint 2 best is to collect 0 geodes with 0 minutes left
quality = 0
final state is (1, 3, 2, 0, 4, 15, 10, 0, 0)
for blueprint 3 best is to collect 1 geodes with 0 minutes left
quality = 3
final state is (2, 4, 4, 1, 2, 4, 6, 1, 0)
for blueprint 4 best is to collect 4 geodes with 0 minutes left
quality = 16
final state is (2, 3, 4, 2, 3, 7, 4, 4, 0)
for blueprint 5 best is to collect 1 geodes with 0 minutes left
quality = 5
final state is (3, 11, 3, 1, 4, 11, 4, 1, 0)
for blueprint 6 best is to collect 10 geodes with 0 minutes left
quality = 60
final state is (3, 10, 6, 3, 4, 10, 12, 10, 0)
for blueprint 7 best is to collect 0 geodes with 0 minutes left
quality = 0
final state is (3, 5, 2, 0, 3, 15, 12, 0, 0)
for blueprint 8 best is to collect 5 geodes with 0 minutes left
quality = 40
final state is (3, 6, 5, 2, 3, 6, 8, 5, 0)
for blueprint 9 best is to collect

In [12]:
max_minutes = 32
initial_state = (1,0,0,0,0,0,0,0, max_minutes)
reached_states = []
ans = 1
blueprints_bests = {}
for blueprint_id, costs in tqdm(blueprints.items()):

    if blueprint_id>3:
        break
    best, best_state = solve(costs, initial_state=initial_state)

    blueprints_bests[blueprint_id] = (best, best_state)
    ans = ans*best[0]
        
print(f'RESULT : {ans}')
for blueprint_id, (best,best_state) in tqdm(blueprints_bests.items()):
    print(f'for blueprint {blueprint_id} best is to collect {best[0]} geodes with {best[1]} minutes left')
    print(f'final state is {best_state}')

RESULT : 10962


100%|██████████| 3/3 [00:00<00:00, 32682.89it/s]

for blueprint 1 best is to collect 58 geodes with 0 minutes left
final state is (3, 3, 7, 10, 3, 8, 7, 58, 0)
for blueprint 2 best is to collect 9 geodes with 0 minutes left
final state is (4, 10, 9, 3, 4, 23, 9, 9, 0)
for blueprint 3 best is to collect 21 geodes with 0 minutes left
final state is (3, 7, 10, 5, 4, 16, 10, 21, 0)
